# Predict Student Performance

In [25]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set variable

In [26]:
PIPELINE_NAME = "student-performance-pipeline"
SCHEMA_PIPELINE_NAME = "student-performance-tfdv-schema"

# Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# from absl import logging
# logging.set_verbosity(logging.INFO)

DATA_ROOT = "data"

In [27]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Pengolahan Data

### Data Ingestion

In [28]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [29]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [30]:
interactive_context.show(example_gen.outputs['examples'])

### Data Validation

#### Summary Statistic

In [31]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [32]:
interactive_context.show(statistics_gen.outputs["statistics"])

#### Data Schema

In [33]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [34]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Attendance (%)',FLOAT,required,,-
'Grades',FLOAT,required,,-
'Sleep Hours',FLOAT,required,,-
'Socioeconomic Score',FLOAT,required,,-
'Study Hours',FLOAT,required,,-


#### Anomali pada dataset

In [35]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [36]:
interactive_context.show(example_validator.outputs['anomalies'])

### Data Preprocessing

In [37]:
TRANSFORM_MODULE_FILE = "student_performance_transform.py"

In [38]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
    
def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}

    attendance = inputs['Attendance (%)']
    outputs['normalized_attendance'] = tft.scale_to_z_score(attendance)

    grades = inputs['Grades']
    outputs['normalized_grades'] = tft.scale_to_0_1(grades)

    sleep_hours = inputs['Sleep Hours']
    outputs['normalized_sleep_hours'] = tft.scale_to_0_1(sleep_hours)

    socioeconomic_score = inputs['Socioeconomic Score']
    outputs['normalized_socioeconomic_score'] = tft.scale_to_0_1(socioeconomic_score)

    study_hours = inputs['Study Hours']
    outputs['normalized_study_hours'] = tft.scale_to_0_1(study_hours)
    
    return outputs

Overwriting student_performance_transform.py


In [39]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmpznvbyaxv
running install
running install_lib


/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


copying build/lib/student_performance_trainer.py -> /tmp/tmpznvbyaxv
copying build/lib/student_performance_transform.py -> /tmp/tmpznvbyaxv
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpznvbyaxv/tfx_user_code_Transform-0.0+c125e6d6ece433c8f015db47b00e7f6a2cf0fd9bcef2eb0f79f365f5b6c35fdb-py3.9.egg-info
running install_scripts
creating /tmp/tmpznvbyaxv/tfx_user_code_Transform-0.0+c125e6d6ece433c8f015db47b00e7f6a2cf0fd9bcef2eb0f79f365f5b6c35fdb.dist-info/WHEEL
creating '/tmp/tmp8cbdgx_u/tfx_user

INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/b212e027da674f0a80f807b8dec99885/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/b212e027da674f0a80f807b8dec99885/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

## Pengembangan dan Validasi Model

### Model

In [40]:
TRAINER_MODULE_FILE="student_performance_trainer.py"

In [41]:
%%writefile {TRAINER_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam.impl as tft_beam
from tfx.components.trainer.fn_args_utils import FnArgs
from tensorflow_metadata.proto.v0 import schema_pb2
from tfx_bsl.public import tfxio
from tfx.examples.penguin import penguin_utils_base as base

# Feature keys
FEATURE_KEYS = [
    'normalized_attendance',
    'normalized_sleep_hours',
    'normalized_socioeconomic_score',
    'normalized_study_hours'
]

LABEL_KEY = 'normalized_grades'

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, tf_transform_output, num_epochs, batch_size=32) -> tf.data.Dataset:
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=LABEL_KEY)

    return dataset

def build_model():
    """Membangun model Keras."""
    inputs = {key: tf.keras.layers.Input(shape=(1,), name=key) for key in FEATURE_KEYS}
    x = tf.keras.layers.Concatenate()(list(inputs.values()))
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    return model

def run_fn(fn_args: FnArgs):
    """Fungsi utama Trainer untuk melatih model."""
    
    # Membangun model
    model = build_model()

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Membaca dataset
    train_dataset = input_fn(fn_args.train_files, tf_transform_output, 10)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output, 10)

    # Callback untuk menyimpan model terbaik
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        fn_args.serving_model_dir,
        save_best_only=True
    )

    # Melatih model
    model.fit(
        train_dataset,
        validation_data=eval_dataset,
        epochs=10,
        callbacks=[model_checkpoint]
    )

    # Menyimpan model akhir
    model.save(fn_args.serving_model_dir, save_format='tf')


Overwriting student_performance_trainer.py


In [42]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE), # Path ke file Python yang berisi fungsi run_fn
    examples=transform.outputs['transformed_examples'], # Contoh dari komponen ExampleGen atau Transform
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'], # Skema data
    train_args=trainer_pb2.TrainArgs(splits=['train']), # Training arguments (misalnya, num_steps)
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),# Evaluation arguments (misalnya, num_steps)
)

interactive_context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying student_performance_trainer.py -> build/lib
copying student_performance_transform.py -> build/lib
installing to /tmp/tmp624ytuq1
running install
running install_lib
copying build/lib/student_performance_trainer.py -> /tmp/tmp624ytuq1
copying build/lib/student_performance_transform.py -> /tmp/tmp624ytuq1
running install_egg_info


/home/ardi/miniconda3/envs/ml-predict-student-performance/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()


running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp624ytuq1/tfx_user_code_Trainer-0.0+b3fc5b37e702cc80952a1878a69e70b9c056b3452381eb99ef2c8cd6270032c5-py3.9.egg-info
running install_scripts
creating /tmp/tmp624ytuq1/tfx_user_code_Trainer-0.0+b3fc5b37e702cc80952a1878a69e70b9c056b3452381eb99ef2c8cd6270032c5.dist-info/WHEEL
creating '/tmp/tmpfnifyz8z/tfx_user_code_Trainer-0.0+b3fc5b37e702cc80952a1878a69e70b9c056b3452381eb99ef2c8cd6270032c5-py3-none-any.whl' and adding '/tmp/tmp624ytuq1' to it
adding 'student_performance_trainer.py'
adding '

Processing ./pipelines/student-performance-pipeline/_wheels/tfx_user_code_Trainer-0.0+b3fc5b37e702cc80952a1878a69e70b9c056b3452381eb99ef2c8cd6270032c5-py3-none-any.whl
Epoch 1/10
    334/Unknown - 2s 4ms/step - loss: 0.0090 - mae: 0.0691INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


...

344/344 [==============================] - 3s 9ms/step - loss: 5.1495e-04 - mae: 0.0175 - val_loss: 6.2131e-04 - val_mae: 0.0192
Epoch 10/10
341/344 [============================>.] - ETA: 0s - loss: 4.7437e-04 - mae: 0.0169INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


344/344 [==============================] - 4s 10ms/step - loss: 4.7405e-04 - mae: 0.0169 - val_loss: 5.4403e-04 - val_mae: 0.0176
INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/student-performance-pipeline/Trainer/model/6/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

### Analisis dan Validasi Model

#### Komponen Resolver

In [43]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

#### Komponen Evaluator

In [44]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='normalized_grades')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [45]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [46]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'mae': {'dou…

## Model Deployment

In [48]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 

pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/predict-student-performance-model'))

)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))

## Test Model

In [52]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/predict-student-performance-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1740127692'}]}
